In [ ]:
from collections import Counter
import pandas as pd
import math
from pprint import pprint

def entropy_of_list(list):
    cnt = Counter(x for x in list)
    n= len(list)*1.0
    probs = [x/n for x in cnt.values()]
    return sum([-x*math.log(x,2) for x in probs])

def information_gain(df,split,target):
    df_split = df.groupby(split)
    df_arg_ent = df_split.agg({target:[entropy_of_list,lambda x:len(x)/len(df)]})
    df_arg_ent.columns=['Entropy','PropObervation']
    new_entorpy= sum(df_arg_ent['Entropy']* df_arg_ent['PropObervation'])
    old_entropy = entropy_of_list(df[target])
    return old_entropy-new_entorpy
    
def id3(df,target,attribute):
    cnt = Counter(x for x in df[target])
    
    if len(cnt)==1:
        return next(iter(cnt))
    
    if df.empty or (not attribute):
        return None
    else:
        gain = [information_gain(df,attr,target) for attr in attribute]
        best_attr=attribute[gain.index(max(gain))]
        tree = {best_attr:{}}
        remaining = [i for i in attribute if i!=best_attr]
        
        for attr_val,data_subset in df.groupby(best_attr):
            tree[best_attr][attr_val] = id3(data_subset,target,remaining)
            
        return tree

df_tennis = pd.read_csv('../input/playtennis/play_tennis.csv')
attribute = list(df_tennis.columns[0:-1])
tree = id3(df_tennis,'play',attribute)
pprint(tree)


def test(tree, d):
    for k in tree:
        for v in tree[k]:
            if (d[k] == v and isinstance(tree[k][v], dict)):
                test(tree[k][v], d)
            elif (d[k] == v):
                print("Classification: " + tree[k][v])
test(tree,df_tennis.loc[2,:])